In [1]:
import rasterio
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import numpy as np
from sarpy.visualization.remap import density
import pandas as pd
import glob
import os
import tensorflow as tf
from tensorflow.keras.utils import Progbar
from PIL import Image

import tensorflow as tf
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import Progbar

In [2]:
def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))

def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [3]:
def read_npy(f):
#     arr = np.load(f.numpy().decode(), allow_pickle=True)[()]
#     x = arr['img'][0]
#     r = arr['detect_scene_row']
#     c = arr['detect_scene_column']
#     id = arr['scene_id']
#     y = arr['label'][0]
    
    img = rasterio.open(vv_f).read(1)
    img[img==-32768.0] = 0.0
    img[img!=0.0] = img[img!=0.0] - np.amin(img)
    
    
    return x, y, r, c, id

In [4]:
def create_datagen(files, shuffle=False, repeat=False, aug=False, batch_size=1):
    
    ds = tf.data.Dataset.from_tensor_slices(files)

#     ds = ds.cache()
    
    #tokenizer func
    ds = ds.map(lambda f: tf.py_function(func=read_npy,
                                       inp=[f],
                                       Tout=[tf.float32, tf.float32, tf.int64, tf.int64, tf.string]), #tf.int32
                 num_parallel_calls=None)
    
#     ds = ds.prefetch(tf.data.AUTOTUNE)

    return ds

In [5]:
def train_serialize_example(img_vv): #, label, height, width, id, row, col):
    feature = {
        'img_vv'        : _float_feature(img_vv),
#         'img_vh'        : _float_feature(img),
#         'img_height' : _int64_feature(height),
#         'img_width'  : _int64_feature(width),
#         'owiWindSpeed'      : _float_feature(label),
#         'owiWindQuality'   : _float_feature(id),
#         'owiWindDirection' :  _float_feature(row),
#         'owiMask' : _float_feature(col)
#         'bathymetry' : _float_feature(col),   
    }
    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()

In [6]:
vh_files = glob.glob('/tf/media/ryan/bigboi/datasets/xview3/public/src/*p/VH*tif')
vv_files = glob.glob('/tf/media/ryan/bigboi/datasets/xview3/public/src/*p/VV*tif')

In [ ]:
base_path = '/tf/media/ryan/bigboi/datasets/xview3/public/src_tfrec/'

for vv_f, vh_f in zip(vv_files, vh_files):
    
    full_path = base_path+os.path.basename(vv_f[:-10])
    
    if not os.path.exists(full_path):
        os.mkdir(full_path, mode=777)

    #create writer
    writer = tf.io.TFRecordWriter(f'{full_path}/shard.tfrec') 
        
    #get image
    img_vv = rasterio.open(vv_f).read(1)

    example  = train_serialize_example(img_vv=img_vv.ravel())
    
#     example  = train_serialize_example(img=x.numpy().ravel(),
#                                        label=y.numpy().ravel(),
#                                        height=x.numpy().shape[0],
#                                        width=x.numpy().shape[1],
#                                        id=id,
#                                        row=r,
#                                        col=c)

    writer.write(example)

    pbar.add(1)

In [ ]:
base_path = '/tf/media/ryan/bigboi/datasets/xview3/public/src_npz/'

for vv_f, vh_f in zip(vv_files, vh_files):
    
    full_path = base_path+os.path.basename(vv_f[:-10])
    
    if not os.path.exists(full_path):
        os.mkdir(full_path, mode=777)
    
#     print()
    
    img = rasterio.open(vv_f).read(1)
    np.savez_compressed(full_path+'/VV_dB', img, allow_pickle=True)
    
    del img
    
#     img = rasterio.open(vh_f).read(1)
    
    
#     del img
    
    break

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/root/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3427, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-14-12e0c40d0510>", line 13, in <module>
    np.savez_compressed(full_path+'/VV_dB', img, allow_pickle=True)
  File "<__array_function__ internals>", line 6, in savez_compressed
  File "/root/miniconda3/lib/python3.7/site-packages/numpy/lib/npyio.py", line 686, in savez_compressed
    _savez(file, args, kwds, True)
  File "/root/miniconda3/lib/python3.7/site-packages/numpy/lib/npyio.py", line 723, in _savez
    pickle_kwargs=pickle_kwargs)
  File "/root/miniconda3/lib/python3.7/site-packages/numpy/lib/format.py", line 680, in write_array
    fp.write(chunk.tobytes('C'))
  File "/root/miniconda3/lib/python3.7/zipfile.py", line 1129, in write
    data = self._compressor.compress(data)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/root/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3427, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-14-12e0c40d0510>", line 13, in <module>
    np.savez_compressed(full_path+'/VV_dB', img, allow_pickle=True)
  File "<__array_function__ internals>", line 6, in savez_compressed
  File "/root/miniconda3/lib/python3.7/site-packages/numpy/lib/npyio.py", line 686, in savez_compressed
    _savez(file, args, kwds, True)
  File "/root/miniconda3/lib/python3.7/site-packages/numpy/lib/npyio.py", line 723, in _savez
    pickle_kwargs=pickle_kwargs)
  File "/root/miniconda3/lib/python3.7/site-packages/numpy/lib/format.py", line 680, in write_array
    fp.write(chunk.tobytes('C'))
  File "/root/miniconda3/lib/python3.7/zipfile.py", line 1129, in write
    data = self._compressor.compress(data)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

In [ ]:
img = np.load('/tf/media/ryan/bigboi/datasets/xview3/public/src_npz/6e3be36f2c7f8ea5p/VV_dB.npz', allow_pickle=True)

In [11]:
vv_f

'/tf/media/ryan/bigboi/datasets/xview3/public/src/6e3be36f2c7f8ea5p/VV_dB.tif'